In [1]:
import cv2
import numpy as np
import os
import math
from matplotlib import pyplot as plt       # чтобы выводить промежуточные фото в jupyter

# Список всех настроечных параметров/констант
WORK_DIR = 'pass_photos'
TEMP_DIR = 'pass_temp'
# TEST_FILE = 'pass_photos/1.jpeg'
IMG_HEIGHT = 1000            # требуемый размер фото для нормализации всех изображений
IMG_WIDTH = 600              # т.к. в задачу входит прочитать только ФИО, обрезаю серию/номер чтобы не усложнять распознавание
INDENT_LEFT = 200            # обрезаем фото т.к. без него получается лучше разделить фото на куски текста
INDENT_BOTTOM = 260          # обрезаем нижние поля
INDENT_TOP = 40              # обрезаем лишнюю часть паспорта снизу

In [2]:
# Функция для получения списка файлов из каталога с фотографиями (как в task_1 и task_2)
def get_files(directory: str) -> list:
    names = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            names.append(os.path.join(directory, filename))

    return names

In [3]:
# Изменение размера
def scale_image(image, scale):     # принимаем объект изображения OpenCV
    
    # получаем текущий размер, вычисляем искомый и создаем измененное изображение
    height, width = image.shape[0], image.shape[1]
    img_width = int(width * scale)
    img_height = int(height * scale)
    img = cv2.resize(image, (img_width, img_height))
    #img = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_CUBIC)
    
    return img

In [4]:
# Нормализация размеров и вырезка нужной части паспорта для обработки
def normalize_size(image):     # принимаем объект изображения OpenCV
    
    # нормализуем фото к нужному размеру
    old_height = image.shape[0]     # получаем исходную высоту
    resize_scale = IMG_HEIGHT / old_height       # считаем коэффициент масштабирования изображения до требуемого
    img = scale_image(image=image, scale=resize_scale)
    new_width = img.shape[1]      # получаем новую ширину
    
    # обрезаем паспорт до страницы с фото
    x0 = INDENT_LEFT                            # отступ слева, т.к. корочка и фото нам не важны
    y0 = IMG_HEIGHT // 2 + INDENT_TOP           # обрезка сверху, т.к. верхняя страница с местом выдачи нам не важна 
    x1 = new_width if new_width < IMG_WIDTH else IMG_WIDTH     # обрезаем все лишнее справа, если есть разворот с пропиской
    y1 = IMG_HEIGHT - INDENT_BOTTOM
    img = img[y0:y1, x0:x1]              # обресанный кусок изображения
    
    return img

In [5]:
# Подготовка изображений для распознавания текста
def normalize_color(image):         # принимаем объект изображения OpenCV
    
    # обесцвечиваем и пытаемся снизить шум с помощью размытия
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
    else:
        gray = image
    # blur = cv2.GaussianBlur(gray, (5,5), 0)         # коэффициент размытия подобран вручную
    
    
    # одно изображение используем для распознавания блоков текста. очередность преобраозвания найдена методом тыка
    kernel = np.ones((5,5), 'uint8')
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    img_block = cv2.erode(gray, kernel, iterations=1)
    #img_block = cv2.dilate(img_block, kernel, iterations=1)
    _, img_block = cv2.threshold(img_block, 0, 255, cv2.THRESH_OTSU, cv2.THRESH_BINARY_INV)
    img_block = cv2.morphologyEx(img_block, cv2.MORPH_OPEN, kernel, iterations=1)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    
    
    # Grayscale, Gaussian blur, Otsu's threshold
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Morph open to remove noise and invert image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    erosion = cv2.erode(gray, kernel, iterations = 1)
    dilation = cv2.dilate(gray, kernel, iterations = 1)
    invert = 255 - closing
    
    
    
    # Повышение контраста
    if len(image.shape) > 2:
        imghsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        imghsv[:,:,2] = [[max(pixel - 25, 0) if pixel < 190 else min(pixel + 25, 255) for pixel in row] for row in imghsv[:,:,2]]
        contrast = cv2.cvtColor(imghsv, cv2.COLOR_HSV2BGR)
        gray_contrast = cv2.cvtColor(contrast, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
        
    
    # при коэффициенте 3 - лучше распознается Васлевский, при 5 - Соколов и Юмакаева
    img_symbol = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 3, 2)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_TOZERO+cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    

    return img_block, gray      # Возвращаем контрастную картинку с разбивкой на блоки и простое ЧБ изображение

In [11]:
# Выделяем элементы текста из изображения
def search_blocks(image, limit: int):
    
    height, width = image.shape[0], image.shape[1]
    # получаем контуры больших пятен на изображении, внутри которых спрятан текст
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # output = image.copy()      #TODO - можно удалить
    
    print(f'Count of Block counoturs: {len(contours)}')
    blocks = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        # print("R", x, y, w, h, hierarchy[0][idx])
        # hierarchy[i][0]: следующий контур текущего уровня
        # hierarchy[i][1]: предыдущий контур текущего уровня
        # hierarchy[i][2]: первый вложенный элемент
        # hierarchy[i][3]: родительский элемент
        # if hierarchy[0][idx][3] == 0:               # если элемент не является самым крупным
        cv2.rectangle(image, (x, y), (x + w, y + h), (70, 0, 0), 1) # для отображаемой картинки
        if limit < h < height and limit < w < width:    # игнорируем маленькие блоки, а также блок размером с изображение
            block = image[y:y + h, x:x + w]     # вырезаем найденный блок из изображения
            blocks.append((idx, y, h, x, w, block))  # сохраняем габариты и изображение блока в список блоков
            
    #print(blocks)
    return blocks
    # return image

In [38]:
# Режем блок изображения арифметически, если он шире одной высоты символа
def cut_blocks(image):
    height, width = image.shape[0], image.shape[1]
    C = 1.2       # просто коэффициент, рассчитанный на широкие буквы вроде Ж, М, Ш и т.д., чтобы их не делило
    if width < height*C:
        print(f'One symbol is True')
        return [image]
    else:
        print(f'One symbol is FALSE')
        result = []
        y, h, = 0, height      # высота и верхняя точке среза - всегда неизменны
        symbol_count = math.ceil(width / height)    # округляем символы до большего целого
        symbol_width = math.floor(width / symbol_count)   # округляем ширину в пикселях до меньшего целого
        
        # while image.shape[1] > image.shape[0]*C :
        for i in range(symbol_count):
            #symbol_count = round(image.shape[1] / image.shape[0])     # ширину делим на высоту ~ количество символов в блоке
            #symbol_count = math.ceil(image.shape[1] / image.shape[0])
            x = i * symbol_width
            print(f'y = {y}, h = {h}, x = {x}, symbol_width = {x+symbol_width}, width = {width}')
            result.append(image[y:h, x:x+symbol_width])
            print(f'symbol {i} is:\n{result[i]}')
            
        print(f'count of separeted symbols: {len(result)}')
        return result

In [39]:
# Запускаем цикл по всем фото в рабочей папке
passports = get_files(WORK_DIR)
# col = 5    # количество колонок таблицы
words = []
for i, p in enumerate(passports):     # идем по списку путей к изображениям
    temp_dir = os.path.join(TEMP_DIR, str(i))
    if not os.path.exists(temp_dir):
        os.mkdir(temp_dir)
        
    print(f'==== Image {i}.jpg =====')
    image = normalize_size(cv2.imread(p))     # получаем кусок паспорта с ФИО
    img_blocks, img_gray = normalize_color(image=image)
    
    cv2.imwrite(f'{TEMP_DIR}/{i}_blocs.jpg', img_blocks)
    cv2.imwrite(f'{TEMP_DIR}/{i}_symbols.jpg', image)
    
    words = search_blocks(image=img_blocks, limit=15)
    print(f'Count of words: {len(words)}')
    
    # получаем все обнаруженные слова из файла, в котором читаются символы
    try:
        for word in words[-3:]:        # забираем только последние 3 слова, т.к. там содержатся искомые данные
            y, h, x, w = word[1:5]     # из списка обнаруженных блоков текста забираем координаты этих блоков
            img_word = img_gray[y:y + h, x:x + w]     # вырезаем слово по его координатам
            # img_word = image[y:y + h, x:x + w]     # вариант с повышением контраста
            img_word = scale_image(img_word, 8)        # увеличиваем изображение
            cv2.imwrite(os.path.join(temp_dir, f'{word[0]}.jpg'), img_word)   #сохраняем файлы только для контроля
            
            
            word_blocks, word_text = normalize_color(image=img_word)
            #word_blocks, word_text = normalize_color(image=word_text)    # вариант с повышением контраста
            symbols = search_blocks(image=word_blocks, limit=80)
            print(f'Count of symbols: {len(symbols)}')
            
            for symbol in symbols:
                word_dir = os.path.join(temp_dir, str(word[0]))
                if not os.path.exists(word_dir):
                    os.mkdir(word_dir)
                
                y, h, x, w = symbol[1:5]
                img_symbol = word_text[y:y + h, x:x + w]
                #cv2.imwrite(os.path.join(word_dir, f'{symbol[0]}-{e}.jpg'), img_symbol)
                
                # Доп. проверка на случай, если буквы плохо отделились
                for e, one_symbol in enumerate(cut_blocks(img_symbol)):
                    print(f'element: {e}, one_symbol: {one_symbol}')
                    cv2.imwrite(os.path.join(word_dir, f'{symbol[0]}-{e}.jpg'), one_symbol)
                    
            
            
            
    except IndexError:             # хотя сейчас Питон не выдает исключение, даже если в списке всего 1 элемент
        print("В паспорте найдено недостаточно данных. Попробуйте сделать более качественный скан.")
    
    
    #plt.subplot(len(passports)//col+1, col, i+1)    # высчитываем высоту таблицы от количества колонок и общего размера списка
    #plt.imshow(image, 'gray')
    # plt.title(p)
    #plt.xticks([]),plt.yticks([])
#plt.show()

==== Image 0.jpg =====
Count of Block counoturs: 13
Count of words: 7
Count of Block counoturs: 7
Count of symbols: 3
One symbol is True
element: 0, one_symbol: [[193 195 198 ... 214 214 214]
 [190 192 195 ... 214 214 214]
 [186 189 192 ... 214 214 214]
 ...
 [201 202 203 ... 199 199 198]
 [203 204 205 ... 199 199 199]
 [205 206 207 ... 200 199 199]]
One symbol is FALSE
y = 0, h = 135, x = 0, symbol_width = 117, width = 235
symbol 0 is:
[[215 215 216 ... 190 190 189]
 [215 215 215 ... 185 186 186]
 [214 215 215 ... 181 181 182]
 ...
 [192 190 187 ... 188 193 197]
 [194 192 190 ... 190 195 199]
 [196 195 192 ... 193 197 202]]
y = 0, h = 135, x = 117, symbol_width = 234, width = 235
symbol 1 is:
[[189 189 189 ... 197 198 200]
 [185 185 185 ... 193 195 198]
 [182 181 181 ... 190 192 195]
 ...
 [202 206 207 ... 214 213 213]
 [204 207 209 ... 214 214 213]
 [206 209 210 ... 214 214 214]]
count of separeted symbols: 2
element: 0, one_symbol: [[215 215 216 ... 190 190 189]
 [215 215 215 ... 18

Count of Block counoturs: 17
Count of words: 6
Count of Block counoturs: 15
Count of symbols: 7
One symbol is True
element: 0, one_symbol: [[159 160 160 ... 171 170 170]
 [158 159 160 ... 172 171 171]
 [158 158 159 ... 172 172 172]
 ...
 [167 168 169 ... 171 170 169]
 [168 169 170 ... 172 170 169]
 [169 170 171 ... 172 171 170]]
One symbol is FALSE
y = 0, h = 116, x = 0, symbol_width = 112, width = 225
symbol 0 is:
[[156 155 154 ... 160 160 160]
 [155 153 152 ... 160 160 159]
 [153 152 150 ... 160 159 158]
 ...
 [151 149 147 ... 120 120 119]
 [154 152 150 ... 127 127 126]
 [157 155 153 ... 134 134 133]]
y = 0, h = 116, x = 112, symbol_width = 224, width = 225
symbol 1 is:
[[160 159 159 ... 164 164 163]
 [159 158 157 ... 164 164 163]
 [158 157 156 ... 164 164 163]
 ...
 [119 118 118 ... 138 140 142]
 [126 125 125 ... 142 144 146]
 [133 133 132 ... 146 147 149]]
count of separeted symbols: 2
element: 0, one_symbol: [[156 155 154 ... 160 160 160]
 [155 153 152 ... 160 160 159]
 [153 152 1

Count of Block counoturs: 11
Count of words: 6
Count of Block counoturs: 16
Count of symbols: 9
One symbol is True
element: 0, one_symbol: [[209 209 209 ... 205 205 206]
 [208 209 209 ... 204 205 206]
 [208 208 209 ... 203 204 206]
 ...
 [203 203 203 ... 210 210 211]
 [205 205 204 ... 212 213 213]
 [206 206 206 ... 215 216 216]]
One symbol is True
element: 0, one_symbol: [[204 203 203 ... 210 211 212]
 [203 202 202 ... 210 211 212]
 [202 201 200 ... 210 211 212]
 ...
 [187 186 185 ... 193 195 197]
 [190 189 189 ... 196 197 199]
 [193 193 192 ... 197 199 200]]
One symbol is True
element: 0, one_symbol: [[202 201 199 ... 200 201 202]
 [199 197 195 ... 195 197 198]
 [196 193 190 ... 191 193 195]
 ...
 [196 197 197 ... 188 188 187]
 [198 199 199 ... 191 190 190]
 [200 201 201 ... 193 193 192]]
One symbol is True
element: 0, one_symbol: [[202 200 199 ... 202 205 206]
 [197 195 193 ... 198 202 204]
 [193 190 187 ... 195 199 201]
 ...
 [190 190 190 ... 184 186 187]
 [193 192 192 ... 188 189 1

Count of Block counoturs: 17
Count of words: 7
Count of Block counoturs: 17
Count of symbols: 7
One symbol is True
element: 0, one_symbol: [[190 189 188 ... 190 191 192]
 [190 189 188 ... 190 191 192]
 [189 188 187 ... 190 191 192]
 ...
 [190 192 193 ... 177 179 181]
 [192 194 195 ... 178 180 182]
 [194 195 196 ... 179 181 183]]
One symbol is True
element: 0, one_symbol: [[185 184 183 ... 189 188 188]
 [185 183 181 ... 188 188 188]
 [184 182 180 ... 188 188 187]
 ...
 [180 180 180 ... 201 200 198]
 [181 182 182 ... 201 200 199]
 [183 183 183 ... 202 201 199]]
One symbol is True
element: 0, one_symbol: [[186 185 185 ... 190 192 193]
 [185 184 183 ... 189 191 193]
 [184 183 182 ... 188 190 192]
 ...
 [186 187 187 ... 173 173 173]
 [187 187 188 ... 174 174 174]
 [188 189 189 ... 175 175 175]]
One symbol is True
element: 0, one_symbol: [[201 202 202 ... 190 190 190]
 [201 201 202 ... 190 191 191]
 [200 201 201 ... 191 191 191]
 ...
 [181 179 177 ... 176 177 178]
 [182 180 178 ... 177 178 1

Count of Block counoturs: 15
Count of words: 7
Count of Block counoturs: 14
Count of symbols: 8
One symbol is True
element: 0, one_symbol: [[229 227 224 ... 237 239 241]
 [227 225 222 ... 236 238 240]
 [225 222 219 ... 233 235 238]
 ...
 [242 243 243 ... 221 223 225]
 [243 243 244 ... 225 227 229]
 [243 244 244 ... 228 230 231]]
One symbol is True
element: 0, one_symbol: [[227 226 224 ... 232 233 234]
 [226 224 222 ... 230 231 232]
 [224 222 220 ... 228 229 230]
 ...
 [222 220 219 ... 239 238 239]
 [225 223 222 ... 240 240 241]
 [227 226 225 ... 242 242 242]]
One symbol is True
element: 0, one_symbol: [[236 235 233 ... 249 249 249]
 [236 235 233 ... 249 249 249]
 [236 235 233 ... 249 249 249]
 ...
 [220 217 215 ... 245 245 245]
 [223 220 218 ... 245 245 245]
 [226 224 222 ... 246 246 246]]
One symbol is True
element: 0, one_symbol: [[237 236 236 ... 232 229 227]
 [235 234 234 ... 232 229 227]
 [233 233 232 ... 232 230 227]
 ...
 [233 232 230 ... 245 245 245]
 [233 232 231 ... 245 245 2

Count of Block counoturs: 13
Count of words: 7
Count of Block counoturs: 12
Count of symbols: 6
One symbol is True
element: 0, one_symbol: [[199 198 196 ... 209 208 208]
 [196 194 192 ... 208 208 207]
 [193 190 188 ... 208 207 206]
 ...
 [189 187 186 ... 207 206 205]
 [192 191 189 ... 208 208 207]
 [195 194 193 ... 210 209 208]]
One symbol is FALSE
y = 0, h = 124, x = 0, symbol_width = 114, width = 228
symbol 0 is:
[[208 206 204 ... 211 212 212]
 [207 205 203 ... 209 211 211]
 [207 205 202 ... 208 210 210]
 ...
 [201 201 200 ... 203 203 202]
 [205 205 204 ... 204 204 203]
 [209 209 209 ... 204 204 203]]
y = 0, h = 124, x = 114, symbol_width = 228, width = 228
symbol 1 is:
[[213 214 214 ... 211 211 211]
 [212 213 214 ... 211 212 212]
 [211 212 213 ... 212 212 212]
 ...
 [202 202 201 ... 207 207 207]
 [203 202 202 ... 207 207 208]
 [203 202 202 ... 207 208 208]]
count of separeted symbols: 2
element: 0, one_symbol: [[208 206 204 ... 211 212 212]
 [207 205 203 ... 209 211 211]
 [207 205 2

Count of Block counoturs: 10
Count of words: 2
Count of Block counoturs: 12
Count of symbols: 1
One symbol is FALSE
y = 0, h = 116, x = 0, symbol_width = 85, width = 256
symbol 0 is:
[[183 183 184 ... 172 173 173]
 [183 183 183 ... 170 171 172]
 [182 182 182 ... 168 169 169]
 ...
 [179 178 176 ... 173 174 175]
 [179 178 177 ... 174 175 176]
 [180 179 177 ... 174 175 176]]
y = 0, h = 116, x = 85, symbol_width = 170, width = 256
symbol 1 is:
[[174 175 175 ... 195 196 196]
 [173 173 174 ... 195 196 196]
 [170 170 171 ... 194 195 196]
 ...
 [177 178 179 ... 186 185 185]
 [178 179 180 ... 186 186 186]
 [177 178 180 ... 187 186 186]]
y = 0, h = 116, x = 170, symbol_width = 255, width = 256
symbol 2 is:
[[196 197 197 ... 196 195 195]
 [197 197 197 ... 196 196 195]
 [196 196 197 ... 195 195 195]
 ...
 [186 187 188 ... 194 193 192]
 [186 187 188 ... 193 192 191]
 [187 187 188 ... 193 192 191]]
count of separeted symbols: 3
element: 0, one_symbol: [[183 183 184 ... 172 173 173]
 [183 183 183 ...